In [52]:
import pandas as pd
import json
import gensim.downloader
import numpy as np

In [53]:
f = open("../data/raw/NewsAPI.json", "r").read()
f = json.loads(f)

In [54]:
links = []
date = []
headlines = []

In [55]:
for i in f["articles"]:
    links.append(i["url"])
    date.append(i["publishedAt"][:10])
    headlines.append(i["title"])

In [56]:
data = pd.DataFrame(
    list(zip(date, links, headlines)), columns=["date", "url", "headline"]
)

In [57]:
data

,date,url,headline
0,2023-01-19,https://www.bbc.co.uk/news/science-environment...,Climate change: Invest in technology that remo...
1,2023-01-17,https://gizmodo.com/propane-industry-paying-in...,The Propane Industry Is Paying TV Influencers ...
2,2023-02-01,https://www.reuters.com/business/energy/hydrog...,Hydrogen as a backup for renewables remains a ...
3,2023-01-13,https://gizmodo.com/cop28-uae-sultan-ahmed-al-...,An Oil Exec Will Lead the UN's 2023 Climate Su...
4,2023-02-10,https://knowablemagazine.org/article/technolog...,The rise of green hydrogen in Latin America
...,...,...,...
95,2023-01-20,https://www.reuters.com/business/sustainable-b...,Davos 2023-Fortescue sees renewables overtakin...
96,2023-01-24,https://www.reuters.com/business/environment/i...,Indian industry turns to biomass as capital ba...
97,2023-01-20,https://www.reuters.com/business/energy/shippi...,Shipping still in dark over clean fuel solutio...
98,2023-01-11,https://www.theguardian.com/business/2023/jan/...,Tevva begins first high-volume production of e...


In [58]:
data = data.drop_duplicates(subset="headline")

In [59]:
glove_vectors = gensim.downloader.load("glove-wiki-gigaword-50")

In [60]:
def toVector(sentence):
    sentence = sentence.split(" ")
    currList = []
    for i in sentence:
        try:
            curr = glove_vectors[i.lower()]
            currList.append(curr)
        except:
            continue
    returnData = np.mean(currList, axis=0).tolist()
    if isinstance(returnData, list):
        return returnData
    return "delete"

In [61]:
data.to_parquet(
    "../data/intermediate/newswithoutembeds.parquet",
    engine="pyarrow",
    compression="snappy",
)

In [62]:
data

,date,url,headline
0,2023-01-19,https://www.bbc.co.uk/news/science-environment...,Climate change: Invest in technology that remo...
1,2023-01-17,https://gizmodo.com/propane-industry-paying-in...,The Propane Industry Is Paying TV Influencers ...
2,2023-02-01,https://www.reuters.com/business/energy/hydrog...,Hydrogen as a backup for renewables remains a ...
3,2023-01-13,https://gizmodo.com/cop28-uae-sultan-ahmed-al-...,An Oil Exec Will Lead the UN's 2023 Climate Su...
4,2023-02-10,https://knowablemagazine.org/article/technolog...,The rise of green hydrogen in Latin America
...,...,...,...
95,2023-01-20,https://www.reuters.com/business/sustainable-b...,Davos 2023-Fortescue sees renewables overtakin...
96,2023-01-24,https://www.reuters.com/business/environment/i...,Indian industry turns to biomass as capital ba...
97,2023-01-20,https://www.reuters.com/business/energy/shippi...,Shipping still in dark over clean fuel solutio...
98,2023-01-11,https://www.theguardian.com/business/2023/jan/...,Tevva begins first high-volume production of e...


In [63]:
data["embedding"] = data.headline.apply(toVector)

In [64]:
data = data[data.embedding != "delete"]

In [65]:
data.to_parquet(
    "../data/processed/newswithoutembeds.parquet",
    engine="pyarrow",
    compression="snappy",
)

In [66]:
data

,date,url,headline,embedding
0,2023-01-19,https://www.bbc.co.uk/news/science-environment...,Climate change: Invest in technology that remo...,"[0.24599780142307281, 0.2255280762910843, 0.47..."
1,2023-01-17,https://gizmodo.com/propane-industry-paying-in...,The Propane Industry Is Paying TV Influencers ...,"[0.09569251537322998, 0.06538517028093338, 0.3..."
2,2023-02-01,https://www.reuters.com/business/energy/hydrog...,Hydrogen as a backup for renewables remains a ...,"[0.2852407395839691, 0.3096398413181305, 0.561..."
3,2023-01-13,https://gizmodo.com/cop28-uae-sultan-ahmed-al-...,An Oil Exec Will Lead the UN's 2023 Climate Su...,"[0.36459553241729736, 0.35237589478492737, 0.1..."
4,2023-02-10,https://knowablemagazine.org/article/technolog...,The rise of green hydrogen in Latin America,"[0.23019738495349884, 0.5426539778709412, -0.0..."
...,...,...,...,...
95,2023-01-20,https://www.reuters.com/business/sustainable-b...,Davos 2023-Fortescue sees renewables overtakin...,"[-0.031586214900016785, 0.3879581391811371, 0...."
96,2023-01-24,https://www.reuters.com/business/environment/i...,Indian industry turns to biomass as capital ba...,"[-0.007447361946105957, -0.046865202486515045,..."
97,2023-01-20,https://www.reuters.com/business/energy/shippi...,Shipping still in dark over clean fuel solutio...,"[0.1473322957754135, 0.1962924748659134, 0.323..."
98,2023-01-11,https://www.theguardian.com/business/2023/jan/...,Tevva begins first high-volume production of e...,"[0.18662670254707336, -0.03912357613444328, -0..."
